In [ ]:
import numpy as np
import pandas as pd 
import time
import requests
from lxml.html import fromstring
from fake_useragent import UserAgent

ua = UserAgent()
headers = {'User-Agent' : ua.random}

In [ ]:
result = pd.read_csv('../데이터/news_크롤링.csv')
result

# 제목 및 본문 크롤링

In [ ]:
def get_information(url, headers, news_type):
    
    if news_type == 'sports':
        title_xpath = '//*[@id="content"]/div/div[1]/div/div[1]/h4'
        text_xpath = '//*[@id="newsEndContents"]/text()'
        
    elif news_type == 'normal_news':
        title_xpath = '//*[@id="articleTitle"]'
        text_xpath = '//*[@id="articleBodyContents"]/text()'
        
    elif news_type == 'etc':
        title_xpath = '//*[@id="content"]/div[1]/div/h2'
        text_xpath = '//*[@id="articeBody"]/text()'
        
    res = requests.get(url, headers=headers)
    parser = fromstring(res.text)

    title = parser.xpath(title_xpath)[0].text_content()
    text = ''.join(parser.xpath(text_xpath))  
    
    return [title, text]

In [ ]:
print(f'전체 url 개수: {result.shape[0]}개')
start = time.time()

for idx in range(result.shape[0]):
    
    url = result.loc[idx, 'url']
    
    try:
        try: 
            result.loc[idx, ['제목', '내용']] = get_information(url, headers, 'normal_news') # 일반뉴스

        except:
            try: 
                result.loc[idx, ['제목', '내용']] = get_information(url, headers, 'sports') # 스포츠뉴스

            except: 
                result.loc[idx, ['제목', '내용']] = get_information(url, headers, 'etc') #연예 뉴스 등 기타 형태의 뉴스
            
    except: 
        print(f'=== {idx+1}번째 {url} 에러 발생 ===') # 링크 수집 후 닫힌 url에 대한 대처

    if (idx+1) % 10 == 0:
        print(f'=== {idx+1}번째 url까지 완료 === 경과시간: {round(time.time() - start)}초') #log
        
result.to_csv('../데이터/네이버뉴스_크롤링.csv', index=False)